# antlr




## introduction



Why do this. Ultimately, it creates a shortcut for including interpreters for any language in these notebooks. Less reliance on other code, and also converting a library from one framework to another.



In [ ]:
// placeholder for readme.md?


## multilungual abstract syntax trees

This is great day for translating between languages.


### select antlr tree

Using antlr templates because look at all the glorious grammars posted right there on it's main Github page! https://github.com/antlr/grammars-v4



#### the code

select antlr tree?


In [ ]:
var fs = require('fs');
var {CommonTokenStream, InputStream} = require('antlr4ts');
var importer = require('../Core')
var {
    selectDom,
    evaluateDom,
    treeToHtml,
    htmlToTree
} = importer.import(['select tree', 'tree to html', 'html to tree'])
var getParser = importer.import('get antlr tool')

//TODO: fix this with own transpiler because no one is going to share their hard work on StringTemplate4 code
function toString(ctx, subctx) {
    var ast = (subctx || ctx)
    if(typeof ast.ownerDocument !== 'undefined') {
        ast = htmlToTree(ast)
    }
    //escodegen.attachComments(ast, ast.comments, ast.tokens, ast.whitespace)
    return escodegen.generate(ast, {
        comment: true,
        tokens: true,
        whitespace: true,
        format: {indent: {style: '    '}, quotes: 'single'}
    })
}

function selectCode(descriptor, code, language) {
    if(typeof code === 'function') {
        code = code.toString()
    }
    if(typeof code === 'string') {
        if(fs.existsSync(code)) {
            code = fs.readFileSync(code).toString('utf8')
            if(typeof language === 'undefined') {
                // TODO: use language file extension from media-server code module gheshi support
            //    language = 
            }
        }
        var parser = getParser(language)
        try {
            const lexer = new lexer(new InputStream(input))
            const parser = new parser(new CommonTokenStream(lexer))
            parser.buildParseTrees = true;
            code = parser(code)
            return code
        } catch(e) {
            console.log(e)
            throw new Error('Could not parse \n' + code)
        }
    }
    // No more selectTree because jsel and friends are inconsistent/partially implemented.
    if(typeof code.evaluate === 'undefined'
       && typeof code.ownerDocument === 'undefined') {
        code = selectDom('//Program', treeToHtml(code))
        code.ownerDocument.toString = toString.bind(code, code)
    }
    return selectDom(descriptor, code)
}

module.exports = {
    selectCode
}


#### TODO: test select jison on some quake 3 C code



In [ ]:
var antlr = require('antlr4ts')
var importer = require('../Core')
var {selectCode} = importer.import('select antlr tree')

var tree = selectCode('*', '/Users/briancullinan/planet_quake/code/q3_ui/ui_cdkey.c')

var root = tree.translationunit()
//console.log(tree.translationunit())
//console.log(tree.ruleNames)
var result = root.accept(new GenericVisitor())
console.log(result[0][0][0][0][0][0][0][0][0][0][0][0][0][1][0][0][0])
//console.log(Object.getPrototypeOf(root.declarationseq()))
//console.log(root.getChild(0).declaration().functiondefinition())


## antlr tools

Creating lexer and parser from .g4 files. Converting XPath to trees. Convert antlr Tool classes to nodejs so we can load grammars on the fly.



### get antlr language



#### the code

get antlr language?

get antlr tool?


In [ ]:

function returnLanguageTools({lexer, parser, visitor, listener}) {
    var result = {
        ...lexer,
        ...parser,
        ...visitor,
        ...listener,
    }
    //result[Object.keys(lexer)[0]] = Object.values(lexer)[0]
    //result[Object.keys(parser)[0]] = Object.values(parser)[0]
    //result[Object.keys(visitor)[0]] = Object.values(visitor)[0]
    //result[Object.keys(listener)[0]] = Object.values(listener)[0]
    return result
}

function getParser(language) {
    // TODO: get parser by language lookup
    if(language === 'cpp' || language === 'c++') {
        var lexer = require('../Resources/Parsers/cpp/CPP14Lexer.ts')
        var parser = require('../Resources/Parsers/cpp/CPP14Parser.ts')
        var visitor = require('../Resources/Parsers/cpp/CPP14Visitor.ts')
        var listener = require('../Resources/Parsers/cpp/CPP14Listener.ts')
        return returnLanguageTools({
            lexer,
            parser,
            visitor,
            listener
        })
    }
}

module.exports = getParser



### antlr tree visitor



#### the code

antlr tree?

antlr tree visitor?


In [ ]:

function typeToString(node) {
    if(node === 'object' && node) {
        return Object.getPrototypeOf(node).constructor.name;
    } else {
        return typeof node;
    }
}

function getGenericToken(child, parser) {
    return {
        type: parser.ruleNames[child.getSymbol().type],
        value: child.getText(),
        range: [child.getSymbol().start, child.getSymbol().stop]
    }
}

function getGenericVisitor({parser, visitor}) {
    function GenericVisitor() {
        visitor.call(this);
        return this;
    }
    
    console.log(parser)
    
    GenericVisitor.prototype = Object.create(visitor.prototype);
    GenericVisitor.prototype.constructor = GenericVisitor;

    GenericVisitor.prototype.visitChildren = function(ctx) {
        if (!ctx) {
          return
        }

        if (ctx.children) {
            return ctx.children.map(child => {
                if (child.children && child.children.length != 0) {
                    return child.accept(this);
                } else {
                    return getGenericToken(child, parser)
                }
            })
        }
    }
    
    return {
        GenericVisitor
    }
}

module.exports = getGenericVisitor


#### TODO: use g4 translator to automatically select entry point, because this is silly




#### TODO: convert antlr tree to json matching esprima format with text locations and tokens lists




In [ ]:
var {TerminalNodeImpl} = require('antlr')


function typeToString(node) {
    if(jsType === 'object' && node) {
        return Object.getPrototypeOf(node).constructor.name;
    } else {
        return typeof node;
    }
}

// TODO: use transpiler to convert a tree from grammer spec to esprima style
function toAntlrTree(parseTree) {
    if(parseTree instanceof TerminalNodeImpl) {
        
    } else if (parseTree instanceof CommonToken) {
        
    } else if (parseTree instanceof CompilationUnitContext) {
        var result = {}
        result['type'] = 'Program' //typeToString(parseTree)
        result['range'] = [0, parseTree.EOF().getSymbol().start]
        var children = []
        for(var i = 0; i < parseTree.getChildCount(); i++) {
            children.push(toJson(parseTree.getChild(i)))
        }
        result['body'] = children
        return result
    }
}

/*
public class Examples {

  private static final Gson PRETTY_PRINT_GSON = new GsonBuilder().setPrettyPrinting().create();
  private static final Gson GSON = new Gson();

  public static String toJson(ParseTree tree) {
    return toJson(tree, true);
  }

  public static String toJson(ParseTree tree, boolean prettyPrint) {
    return prettyPrint ? PRETTY_PRINT_GSON.toJson(toMap(tree)) : GSON.toJson(toMap(tree));
  }

  public static Map<String, Object> toMap(ParseTree tree) {
    Map<String, Object> map = new LinkedHashMap<>();
    traverse(tree, map);
    return map;
  }

  public static void traverse(ParseTree tree, Map<String, Object> map) {

    if (tree instanceof TerminalNodeImpl) {
      Token token = ((TerminalNodeImpl) tree).getSymbol();
      map.put("type", token.getType());
      map.put("text", token.getText());
    }
    else {
      List<Map<String, Object>> children = new ArrayList<>();
      String name = tree.getClass().getSimpleName().replaceAll("Context$", "");
      map.put(Character.toLowerCase(name.charAt(0)) + name.substring(1), children);

      for (int i = 0; i < tree.getChildCount(); i++) {
        Map<String, Object> nested = new LinkedHashMap<>();
        children.add(nested);
        traverse(tree.getChild(i), nested);
      }
    }
  }

  public static void main(String[] args) {
    String source = "(1 + 2) * 3";
    ExpressionLexer lexer = new ExpressionLexer(CharStreams.fromString(source));
    ExpressionParser parser = new ExpressionParser(new CommonTokenStream(lexer));
    System.out.println(toJson(parser.parse()));
  }
}
*/

module.export = toAntlrTree


#### lets not forget what an esprima tree looks


In [ ]:
var importer = require('../Core')
var {selectAst} = importer.import('select code tree')
var {htmlToTree} = importer.import('html to tree')

var root = selectAst('//Program', importer.interpret('select antlr tree').code)
console.log(htmlToTree(root))

/*

expects
{
  type: 'Program',
  body: [
    {
      type: 'VariableDeclaration',
      declarations: [Array],
      kind: 'var',
      range: [Array]
    },
    {
      type: 'VariableDeclaration',
      declarations: [Array],
      kind: 'var',
      range: [Array]
    },
  ],
  sourcetype: 'script',
  range: [ 0, 2317 ],
  comments: [
    {
      type: 'Line',
      value: ' TODO: get parser by language lookup',
      range: [Array]
    },
  ],
  tokens: [
    { type: 'Keyword', value: 'var', range: [Array] },
    { type: 'Identifier', value: 'fs', range: [Array] },
    { type: 'Punctuator', value: '=', range: [Array] },
  ]
}
*/

